In [ ]:
!python -m venv .venv

In [6]:
!pwd

/home/chisphung/ALPR_Fisheye/notebooks


In [7]:
%cd ..

/home/chisphung/ALPR_Fisheye


In [8]:
import argparse
import os
import sys
from model.yolo.ultralytics.models.yolo.model import YOLO

ModuleNotFoundError: No module named 'ultralytics'

In [5]:


YOLO('yolov11x.pt').train(data='configs/alpr_det.yaml', epochs=100, imgsz=640, batch=16, name='yolov11x_alpr_det_100e')



NameError: name 'YOLO' is not defined